In [1]:
import pandas as pd

In [4]:
df_ita = pd.read_parquet("../01_Prepare Data/ITA_raw.parquet").set_index(
    ["SkinID", "Region", "Pigment", "Side"]
)
df = pd.read_excel("SummaryDetails.xlsx")
df = df.rename(columns={"Fitzpatrick Type": "Group", "Gender": "Sex"})
df_ita = df_ita.loc[
    (slice(None, None), "Carotid Artery") + (slice(None, None),) * 2, "ITA"
].reset_index()[
    ["SkinID", "ITA"]
]  # .groupby("SkinID").()

In [5]:
df = df.merge(df_ita, on="SkinID")

In [6]:
df = df.groupby(["SkinID", "Age", "Sex", "Group"]).mean().reset_index()

In [7]:
def format_mean_std(x):
    m = x.mean()
    s = x.std()
    return rf"{m:.1f} $\pm$ {s:.1f}"


def format_range(x):
    mx = x.max()
    mn = x.min()
    return rf"({mn:.1f}, {mx:.1f})"


def format_gender(x):
    vc = x.value_counts()
    return f"{vc['Male']} M, {vc['Female']} F"

In [8]:
df

,SkinID,Age,Sex,Group,BMI,ITA
0,SKIN01,27,Male,2,25.5,54.196000
1,SKIN02,31,Male,2,29.9,38.540000
2,SKIN03,43,Male,3,27.5,17.108000
3,SKIN04,25,Female,3,22.0,30.570000
4,SKIN05,27,Female,3,21.5,36.326000
5,SKIN06,73,Female,2,21.6,26.894000
6,SKIN07,32,Male,3,27.7,33.540000
7,SKIN08,35,Male,4,24.6,-20.208000
8,SKIN09,57,Female,4,21.9,7.684000
9,SKIN10,65,Female,2,26.6,38.072000


In [9]:
df_summary = (
    df.groupby("Group")
    .agg(
        {
            "Age": [format_mean_std, format_range],
            "Sex": format_gender,
            "BMI": [format_mean_std, format_range],
            "ITA": [format_mean_std, format_range],
        }
    )
    .T
)

df_summary.index.rename(["", ""], inplace=True)
df_summary.index = df_summary.index.set_levels(
    [f"DELETE_COLUMN_{i}" for i in range(df_summary.shape[0])], level=1
)  # ,inplace=True)
df_summary

Group                              1                2               3  \
                                                                        
Age DELETE_COLUMN_1  53.5 $\pm$ 22.2  46.2 $\pm$ 21.4  31.7 $\pm$ 6.4   
    DELETE_COLUMN_2     (24.0, 76.0)     (24.0, 73.0)    (25.0, 43.0)   
Sex DELETE_COLUMN_0         1 M, 5 F         3 M, 3 F        2 M, 4 F   
BMI DELETE_COLUMN_1   21.2 $\pm$ 1.4   24.5 $\pm$ 4.0  23.6 $\pm$ 3.3   
    DELETE_COLUMN_2     (19.1, 23.1)     (18.5, 29.9)    (19.7, 27.7)   
ITA DELETE_COLUMN_1   44.6 $\pm$ 4.5  36.6 $\pm$ 11.3  28.8 $\pm$ 6.8   
    DELETE_COLUMN_2     (39.1, 52.0)     (22.0, 54.2)    (17.1, 36.3)   

Group                              4                 5                 6  \
                                                                           
Age DELETE_COLUMN_1  34.0 $\pm$ 12.9   37.5 $\pm$ 15.2    34.0 $\pm$ 9.5   
    DELETE_COLUMN_2     (22.0, 57.0)      (23.0, 65.0)      (26.0, 48.0)   
Sex DELETE_COLUMN_0         3 M, 3 F          3 M, 3 F          4 M, 2 F   
BMI DELETE_COLUMN_1   23.3 $\pm$ 2.6    23.8 $\pm$ 2.8    23.8 $\pm$ 3.2   
    DELETE_COLUMN_2     (19.6, 27.2)      (20.2, 27.5)      (19.6, 27.5)   
ITA DELETE_COLUMN_1   2.2 $\pm$ 14.6  -20.5 $\pm$ 20.8  -52.7 $\pm$ 12.8   
    DELETE_COLUMN_2    (-20.2, 23.5)      (-59.2, 0.1)    (-67.3, -34.0)   

Group                       Vitiligo  
                                      
Age DELETE_COLUMN_1  59.7 $\pm$ 12.7  
    DELETE_COLUMN_2     (40.0, 72.0)  
Sex DELETE_COLUMN_0         2 M, 4 F  
BMI DELETE_COLUMN_1   25.0 $\pm$ 4.1  
    DELETE_COLUMN_2     (20.2, 28.7)  
ITA DELETE_COLUMN_1  39.1 $\pm$ 13.3  
    DELETE_COLUMN_2     (14.0, 51.2)

In [10]:
def format_columns(x):
    if x == 1:
        return "Fitzpatrick I"
    elif x != "Vitiligo":
        return "FP " + str(x)
    else:
        return x


df_summary.columns = list(map(format_columns, df_summary.columns))
df_summary = df_summary.rename(index={"BMI": "BMI (kg/m2)", "ITA": "ITA (degrees)"})
# df_summary.reset_index(level=1,drop=True)

In [11]:
lat = df_summary.to_latex()
for i in range(df_summary.shape[0]):
    lat = lat.replace(f"DELETE_COLUMN_{i} & ", "")
lat = lat.replace(" &  &  &  &  &  &  &  &  \\\\\n", "")
lat = lat.replace("&  & Fitzpatrick I", "& Fitzpatrick I")
lat = lat.replace("lllllllll", "llllllll")
lat = lat.replace(r"\cline{1-9}", "")

In [12]:
print(lat)

\begin{tabular}{llllllll}
\toprule
 & Fitzpatrick I & FP 2 & FP 3 & FP 4 & FP 5 & FP 6 & Vitiligo \\
\midrule
\multirow[t]{2}{*}{Age} & 53.5 $\pm$ 22.2 & 46.2 $\pm$ 21.4 & 31.7 $\pm$ 6.4 & 34.0 $\pm$ 12.9 & 37.5 $\pm$ 15.2 & 34.0 $\pm$ 9.5 & 59.7 $\pm$ 12.7 \\
 & (24.0, 76.0) & (24.0, 73.0) & (25.0, 43.0) & (22.0, 57.0) & (23.0, 65.0) & (26.0, 48.0) & (40.0, 72.0) \\

Sex & 1 M, 5 F & 3 M, 3 F & 2 M, 4 F & 3 M, 3 F & 3 M, 3 F & 4 M, 2 F & 2 M, 4 F \\

\multirow[t]{2}{*}{BMI (kg/m2)} & 21.2 $\pm$ 1.4 & 24.5 $\pm$ 4.0 & 23.6 $\pm$ 3.3 & 23.3 $\pm$ 2.6 & 23.8 $\pm$ 2.8 & 23.8 $\pm$ 3.2 & 25.0 $\pm$ 4.1 \\
 & (19.1, 23.1) & (18.5, 29.9) & (19.7, 27.7) & (19.6, 27.2) & (20.2, 27.5) & (19.6, 27.5) & (20.2, 28.7) \\

\multirow[t]{2}{*}{ITA (degrees)} & 44.6 $\pm$ 4.5 & 36.6 $\pm$ 11.3 & 28.8 $\pm$ 6.8 & 2.2 $\pm$ 14.6 & -20.5 $\pm$ 20.8 & -52.7 $\pm$ 12.8 & 39.1 $\pm$ 13.3 \\
 & (39.1, 52.0) & (22.0, 54.2) & (17.1, 36.3) & (-20.2, 23.5) & (-59.2, 0.1) & (-67.3, -34.0) & (14.0, 51.2) \\

\bott

In [13]:
def formatter(x):
    return f"{x:.1f}"

In [14]:
df_all_participants = df.set_index("SkinID")
df_all_participants["BMI"] = df_all_participants["BMI"].apply(formatter)
df_all_participants["ITA"] = df_all_participants["ITA"].apply(formatter)
df_all_participants = df_all_participants.rename(
    columns={"BMI": "BMI (kg/m2)", "ITA": "ITA (degrees)"}
)

In [15]:
print(df_all_participants.to_latex())

\begin{tabular}{lrllll}
\toprule
 & Age & Sex & Group & BMI (kg/m2) & ITA (degrees) \\
SkinID &  &  &  &  &  \\
\midrule
SKIN01 & 27 & Male & 2 & 25.5 & 54.2 \\
SKIN02 & 31 & Male & 2 & 29.9 & 38.5 \\
SKIN03 & 43 & Male & 3 & 27.5 & 17.1 \\
SKIN04 & 25 & Female & 3 & 22.0 & 30.6 \\
SKIN05 & 27 & Female & 3 & 21.5 & 36.3 \\
SKIN06 & 73 & Female & 2 & 21.6 & 26.9 \\
SKIN07 & 32 & Male & 3 & 27.7 & 33.5 \\
SKIN08 & 35 & Male & 4 & 24.6 & -20.2 \\
SKIN09 & 57 & Female & 4 & 21.9 & 7.7 \\
SKIN10 & 65 & Female & 2 & 26.6 & 38.1 \\
SKIN11 & 57 & Female & 2 & 18.5 & 22.0 \\
SKIN12 & 74 & Female & 1 & 19.1 & 42.1 \\
SKIN13 & 24 & Male & 2 & 25.2 & 40.0 \\
SKIN14 & 32 & Female & 4 & 27.2 & 1.1 \\
SKIN15 & 36 & Female & 4 & 24.2 & 23.5 \\
SKIN16 & 76 & Female & 1 & 20.2 & 39.1 \\
SKIN17 & 29 & Female & 3 & 19.7 & 30.2 \\
SKIN18 & 34 & Female & 3 & 23.2 & 25.4 \\
SKIN19 & 38 & Female & 5 & 27.5 & -59.2 \\
SKIN20 & 59 & Male & 1 & 23.1 & 42.5 \\
SKIN21 & 29 & Female & 1 & 22.0 & 46.2 \\
SKIN22 & 22